#Initialization

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import csv

import pandas as pd
import pickle

In [ ]:
from google.colab import drive, files
# drive.mount('/content/drive')

FILELOC = "/content/drive/My Drive/COVID_Python/"

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    tpu_env=True
except ValueError:
    print('Not connected to a TPU runtime.')
    tpu_env=False

Running on TPU  ['10.11.0.130:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.11.0.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.11.0.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def reset_model(regress, singleclass, multiclass, output_multiheadatt, use_att, nclasses=4):

    if output_multiheadatt:
        model_fn = AttMod_2
    else:
        model_fn = AttModel
    model = model_fn(L=ismlen,
                     vocab_size=len(aa_list)+1,
                     embdim = ENCDIM,
                     numheads = NHEADS,
                     ffdim = FFDIM,
                     num_dense = NDENSE,
                     mask_zero=True,
                     dropout_rate = DROPRATE,
                     trans_drop = TRANSDROPRATE,
                     Nt = NT,
                     W = 1, Nc = NC, Nl = NL,
                     regress=regress, singleclass=singleclass,
                     multiclass=multiclass, use_att=use_att,
                     nclasses=nclasses,
                     )
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARN_RATE)
    if regress:
        loss = keras.losses.MeanSquaredError()
        metrics = [keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.MeanSquaredLogarithmicError(name='msle'),
            keras.losses.MeanAbsoluteError(name='mae')
            ]
    if singleclass:
        loss = keras.losses.BinaryCrossentropy()
        metrics = [keras.metrics.BinaryAccuracy(name='acc')]
    if multiclass:    
        loss = keras.losses.SparseCategoricalCrossentropy()
        metrics = [keras.metrics.SparseCategoricalAccuracy(name='acc')]
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics,)
                #   steps_per_execution = STEPS_PER_EXECUTION,)
    
    return model

In [ ]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, mask_zero=False):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size,
                                                output_dim=embed_dim,
                                                mask_zero=mask_zero)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim,
                                              mask_zero=mask_zero)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def linear01(x):
    return tf.clip_by_value(x, clip_value_min=0, clip_value_max=1)

In [ ]:
def AttMod_2(L, vocab_size, embdim, numheads, ffdim, num_dense=False,
             mask_zero=False, dropout_rate=False, trans_drop=0.1,
             Nt=1, W=False, Nc=False, Nl=False,
             regress=True, singleclass=False, multiclass=False, use_att=True,
             nclasses=4):

    inpTensor = keras.Input(shape=(L,))
    x = inpTensor

    if mask_zero:
        x = keras.layers.Masking(mask_value=0)(x)   

    x = TokenAndPositionEmbedding(L, vocab_size, embdim, mask_zero)(x)

    if W and Nc and Nl:
        for n in range(Nl):
            x = keras.layers.Conv1D(filters = Nc,
                                kernel_size = W,
                                activation = 'relu',
                                padding = 'same',
                                )(x)
            if n > 1 and n < Nl-1:
                x = keras.layers.BatchNormalization()(x)

    y, attout = keras.layers.MultiHeadAttention(num_heads=numheads, key_dim=embdim,
                                                )(x, x, return_attention_scores=True)
    y = keras.layers.Dropout(trans_drop)(y)
    z = keras.layers.LayerNormalization(epsilon=1e-6)(x + y)
    z1 = keras.Sequential( [keras.layers.Dense(ffdim, activation="relu"), keras.layers.Dense(embdim),])
    z1 = keras.layers.Dropout(trans_drop)(z)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(z + z1)

    if use_att:
        # Attention layer
        h = keras.layers.TimeDistributed(keras.layers.Dense(Nc, activation='tanh'))(x)
        attention = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='tanh'))(h)
        attention = keras.layers.Flatten()(attention)  
        attention = keras.layers.Softmax(axis=1, name='attention')(attention) # normalize attention values
        attention = keras.layers.RepeatVector(Nc)(attention)
        attention = keras.layers.Permute([2, 1])(attention)
        representation = keras.layers.multiply([h, attention])
        representation = tf.math.reduce_sum(representation, axis = 1)
        x = representation
    else:
        x = keras.layers.GlobalAveragePooling1D()(x)

    if num_dense:
        x = keras.layers.Dense(num_dense, activation = 'relu')(x)
    if dropout_rate:
        x = keras.layers.Dropout(Params[dropout_rate])(x)

    if regress:
        finalOut = keras.layers.Dense(1, activation=linear01)(x)
    if singleclass:
        finalOut = keras.layers.Dense(1, activation='sigmoid')(x)
    if multiclass:
        finalOut = keras.layers.Dense(nclasses, activation='softmax')(x)

    # define the model's start and end points    
    model = keras.Model(inpTensor,finalOut)

    return model

In [ ]:
def AttModel(L, vocab_size, embdim, numheads, ffdim, num_dense=False,
             mask_zero=False, dropout_rate=False, trans_drop=0.1,
             Nt=1, W=False, Nc=False, Nl=False,
             regress=True, singleclass=False, multiclass=False, use_att=True,
             nclasses=4):

    inpTensor = keras.Input(shape=(L,))
    x = inpTensor

    if mask_zero:
        x = keras.layers.Masking(mask_value=0)(x)   

    x = TokenAndPositionEmbedding(L, vocab_size, embdim, mask_zero)(x)

    if W and Nc and Nl:
        for n in range(Nl):
            x = keras.layers.Conv1D(filters = Nc,
                                kernel_size = W,
                                activation = 'relu',
                                padding = 'same',
                                )(x)
            if n > 1 and n < Nl-1:
                x = keras.layers.BatchNormalization()(x)

    for n in range(Nt):
        x = TransformerBlock(Nc, numheads, ffdim, rate=trans_drop)(x)

    if use_att:
        # Attention layer
        h = keras.layers.TimeDistributed(keras.layers.Dense(Nc, activation='tanh'))(x)
        attention = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='tanh'))(h)
        attention = keras.layers.Flatten()(attention)  
        attention = keras.layers.Softmax(axis=1, name='attention')(attention) # normalize attention values
        attention = keras.layers.RepeatVector(Nc)(attention)
        attention = keras.layers.Permute([2, 1])(attention)
        representation = keras.layers.multiply([h, attention])
        representation = tf.math.reduce_sum(representation, axis = 1)
        x = representation
    else:
        x = keras.layers.GlobalAveragePooling1D()(x)

    if num_dense:
        x = keras.layers.Dense(num_dense, activation = 'relu')(x)
    if dropout_rate:
        x = keras.layers.Dropout(Params[dropout_rate])(x)

    if regress:
        finalOut = keras.layers.Dense(1, activation=linear01)(x)
    if singleclass:
        finalOut = keras.layers.Dense(1, activation='sigmoid')(x)
    if multiclass:
        finalOut = keras.layers.Dense(nclasses, activation='softmax')(x)

    # define the model's start and end points    
    model = keras.Model(inpTensor,finalOut)

    return model

In [ ]:
# These parameters are currently hard-coded
ENCDIM = 1500
NC = 300
NL = 1
NT = 1
NHEADS = 8
FFDIM = 64
NDENSE = 64
TRANSDROPRATE = 0.1
DROPRATE = 0.0

LEARN_RATE = 0.0001

BATCH_SIZE = 48

STEPS_PER_EXECUTION = 50

#Get Omicron spike sequences

In [ ]:
DIR = FILELOC + 'coronavirus_spike/'

In [ ]:
!pip install scikit-bio
import skbio
from skbio import TabularMSA, DNA, Protein
from skbio.alignment import local_pairwise_align_ssw

In [ ]:
seqs = [seq for seq in skbio.io.read(DIR+'gisaid_hcov-19_2021_11_28_08.fasta', format='fasta')]
spike = DNA.read(DIR+'EPI_ISL_402124-S.fasta')  # reference sequence from NCBI
align = [local_pairwise_align_ssw(DNA(seq),spike) for seq in seqs]
# align is a triplet of (alignment, score, start_end_positions)

In [ ]:
# spike.translate()
spike_prot = ''.join([x.decode() for x in spike.translate().values])

In [ ]:
# find aligned spike sequences without degenerates to translate
no_degenerates = [k for k in range(len(seqs)) if not align[k][0][0].has_degenerates()]
omicron_protein_seqs = [align[k][0][0].degap().translate() for k in no_degenerates]
omicron_seqs = [''.join([x.decode() for x in seq.values]) for seq in omicron_protein_seqs]
omicron_metadata = [seq.metadata['id'] for seq in omicron_protein_seqs]

In [ ]:
omicrondf = pd.DataFrame.from_dict({'Metadata':omicron_metadata})
omicrondf['SeqID'] = omicrondf.Metadata.apply(lambda x:x.split('|')[1])
omicrondf['Spike'] = omicron_seqs

In [ ]:
# omicrondf.to_csv(FILELOC + 'omicron_spike_20211128.csv',index=False)

#Predict vaccine resistance

In [ ]:
ismlen = 1273

def tokenize_sequences(data_dataframe, SeqCol='ISM', seqlen=1273):
    def f(x):
        if len(x) < seqlen:
            return x + '*'*(seqlen-len(x))
        elif len(x) > seqlen:
            return x[:seqlen]
        else:
            return x
    data = np.vstack(data_dataframe[SeqCol].apply(f).apply(lambda x: np.array(list(x))))
    aa_list = ['A', 'R', 'N', 'D', 'C', 'Q', 'E',
            'G', 'H', 'I', 'L', 'K', 'M', 'F',
            'P', 'S', 'T', 'W', 'Y', 'V', '-',
            ]
    aa_tokenizer = {aa_list[k]:k+1 for k in range(len(aa_list))}
    aa_tokenizer['*'] = 0
    aa_tokenizer['X'] = 0
    # optionally handle B, J, Z ambiguities
    # Asx	B	Aspartic acid or Asparagine (D or N)
    # Glx	Z	Glutamic acid or Glutamine (E or Q)
    # Xaa	X	Any amino acid
    # Xle	J	Leucine or Isoleucine (L or I)
    aa_tokenizer['B'] = 0
    aa_tokenizer['Z'] = 0
    aa_tokenizer['J'] = 0

    return np.vectorize(aa_tokenizer.get)(data)

aa_list = ['A', 'R', 'N', 'D', 'C', 'Q', 'E',
        'G', 'H', 'I', 'L', 'K', 'M', 'F',
        'P', 'S', 'T', 'W', 'Y', 'V', '-',
        ]

Cromer D, Steain M, Reynaldi A, Schlub TE, Wheatley AK, Juno JA, Kent SJ, Triccas JA, Khoury DS, Davenport MP. Neutralising antibody titres as predictors of protection against SARS-CoV-2 variants and the impact of boosting: a meta-analysis. Lancet Microbe. 2021 Nov 15. doi: 10.1016/S2666-5247(21)00267-6. Epub ahead of print. PMID: 34806056; PMCID: PMC8592563.

https://pubmed.ncbi.nlm.nih.gov/34806056/

"The neutralising activity against the ancestral SARS-CoV-2 was highly predictive of neutralisation of variants of concern. Decreases in neutralisation titre to the alpha (1·6-fold), beta (8·8-fold), gamma (3·5-fold), and delta (3·9-fold) variants (compared to the ancestral virus) were not significantly different between different vaccines."

Arora, P., Kempf, A., Nehlmeier, I. et al. Delta variant (B.1.617.2) sublineages do not show increased neutralization resistance. Cell Mol Immunol 18, 2557–2559 (2021). https://doi.org/10.1038/s41423-021-00772-y

In [ ]:
### Lineage -> Neutralization mapping

lins = ['A', 'A.1', 'A.2.5', 'B', 'B.1', 'B.1.1.7', 'B.1.351', 'B.1.351.2', 'B.1.351.5', 'P.1', 'B.1.617.2']
neutr = [1, 1, 1, 1, 1, 1/1.6, 1/8.8, 1/8.8, 1/8.8, 1/3.5, 1/3.9]
assert len(lins) == len(neutr)
numlins = len(lins)

labelmap = {lins[k]:neutr[k] for k in range(numlins)}

In [ ]:
with open(FILELOC + 'aligned_sequence_lineages_1001.pkl', 'rb') as f:
    _, datadf = pickle.load(f)
datadf.reset_index(drop=False, inplace=True)

# with open(FILELOC + 'raw_sequence_lineages_1001.pkl', 'rb') as f:
#     _, datadf = pickle.load(f)
# datadf.reset_index(drop=False, inplace=True)

In [ ]:
tdf = datadf[datadf.Lineage.isin(lins)]
print([len(tdf[tdf.Count >= c]) for c in range(1,6)])

[275851, 38834, 22883, 16711, 13211]


In [ ]:
MINCOUNT = 10
tdf[tdf.Count >= MINCOUNT]['Lineage'].map(labelmap).value_counts()

0.625000    3535
0.256410    1990
1.000000     573
0.285714     362
0.113636     253
Name: Lineage, dtype: int64

In [ ]:
traindf = tdf[tdf.Count >= MINCOUNT].copy()
traindf['label'] = traindf.Lineage.map(labelmap)

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours

# SRS = 2077

# sm = RandomUnderSampler(sampling_strategy='majoriy', random_state=SRS, replacement=False)
# xt, yt = sm.fit_resample(xtrain, traindf.Lineage.map({lins[ind]:ind for ind in range(len(lins))}))

### Manual undersampling
drop_indices = {}
for label in np.unique(traindf.label):
    n = len(traindf[traindf.label==label])
    if n > 500:
        print(label, len(traindf))
        dropindices[label] = np.random.choice(traindf[traindf.label==label].index, n-500, replace=False)
        traindf.drop(dropindices[label], inplace=True)
        print(len(traindf))

0.25641025641025644 6713
5223
0.625 5223
2188
1.0 2188
2115


In [ ]:
xtrain = tokenize_sequences(traindf, 'MaskedSeq', ismlen)
# xtrain = tokenize_sequences(traindf, 'Spike', ismlen)

ytrain = traindf.Lineage.map(labelmap).values

In [ ]:
NUM_EPOCHS = 200

In [ ]:
tf.keras.backend.clear_session()
with tpu_strategy.scope():
    model = reset_model(regress=True, singleclass=False, multiclass=False,
                        output_multiheadatt=False, use_att=True)
history = model.fit(xtrain, ytrain,
                    batch_size = BATCH_SIZE,
                    epochs = NUM_EPOCHS,
                    verbose = 1,
                    )

In [ ]:
model.save_weights(FILELOC + 'coronavirus_vaxresist_20211128.h5', save_format='h5', overwrite=True)

In [ ]:
ttok = tokenize_sequences(tdf[tdf.Count >= 4], 'MaskedSeq', ismlen)

In [ ]:
seqdf = tdf[tdf.Count >= 4].copy()

In [ ]:
seqdf['label'] = seqdf.Lineage.map(labelmap)

In [ ]:
seqdf['predict'] = model.predict(ttok, verbose=1, batch_size=32)

523/523 [==============================] - 17s 31ms/step


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(ytrain, model.predict(xtrain)))

0.06500075755614033

In [ ]:
np.sqrt(mean_squared_error(ytrain, np.mean(ytrain)*np.ones(len(ytrain))))

0.32013448776184833

In [ ]:
np.sqrt(mean_squared_error(seqdf.label, np.mean(seqdf.label)*np.ones(len(seqdf))))

0.24040585856442648

In [ ]:
np.sqrt(mean_squared_error(seqdf.label, seqdf.predict))

0.06006695525383249

In [ ]:
omicron = pd.read_csv(FILELOC + "omicron_spike_20211128.csv")
otok = tokenize_sequences(omicron,'Spike',ismlen)

In [ ]:
omicron['Predict'] = model.predict(otok)

In [ ]:
1/8.8

0.11363636363636363

In [ ]:
1/np.max(omicron.Predict)

4.724169176973992

In [ ]:
1/np.mean(omicron.Predict)

5.308718477714436